In [8]:
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import urllib.request
from konlpy.tag import Okt



import json

tokenizer = Tokenizer()

# wordInext 목록 받아오기
with open('jsonDataset_wordIndex.json') as f:
    word_index = json.load(f)
    tokenizer.word_index = word_index

model = tf.keras.models.load_model('model_test.h5')
model.summary()

okt = Okt()

def preTreatment(data):
    stopwords = []
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp


def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence]) # 단어를 숫자값, 인덱스로 변환하여 저장
    print(data)
    max_len = 189 # 전체 데이터 셋 길이 설정 (메일의 최대 길이)
    pad_new = pad_sequences(data, maxlen = max_len) # 모든 메일의 길이를 189로 설정 (빈 부분은 0으로 패딩)
    score = float(model.predict(pad_new))
    if (score > 0.5):
        print(' {:.2f}% 확률로 노인남녀입니다.'.format(score * 100))
    else:
        print(' {:.2f}% 확률로 일반남녀입니다.'.format((1 - score) * 100))

txt = ''
data = preTreatment(txt)
sentiment_predict(data)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         1345900   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,463,277
Trainable params: 1,463,277
Non-trainable params: 0
_________________________________________________________________
[[86, 82, 1]]
 96.98% 확률로 일반남녀입니다.
